In [7]:
import os
import pandas as pd
import datetime
import urllib.request
import json
import requests, json

In [8]:
df_station = pd.read_csv('Data_Station.csv')
df_station

,id,location,name
0,02t,แขวงหิรัญรูจี เขตธนบุรี กทม.,มหาวิทยาลัยราชภัฏบ้านสมเด็จเจ้าพระยา
1,05t,แขวงบางนา เขตบางนา กทม.,กรมอุตุนิยมวิทยา
2,10t,แขวงคลองจั่น เขตบางกะปิ กทม.,เคหะชุมชนคลองจั่น
3,11t,แขวงดินแดง เขตดินแดง กทม.,สนามกีฬาเคหะชุมชนห้วยขวาง
4,12t,แขวงช่องนนทรี เขตยานนาวา กทม.,โรงเรียนนนทรีวิทยา
5,59t,แขวงพญาไท เขตพญาไท กทม.,กรมประชาสัมพันธ์
6,61t,แขวงพลับพลา เขตวังทองหลาง กทม.,โรงเรียนบดินทรเดชา (สิงห์ สิงหเสนี)
7,03t,ริมถนนกาญจนาภิเษก แขวงแสมดำ เขตบางขุนเทียน กทม.,ริมถนนกาญจนาภิเษก
8,50t,ริมถนนพระรามสี่ แขวงปทุมวัน เขตปทุมวัน กทม.,โรงพยาบาลจุฬาลงกรณ์
9,52t,ริมถนนอินทรพิทักษ์ แขวงบางยี่เรือ เขตธนบุรี กทม.,สถานีไฟฟ้าย่อยธนบุรี


In [9]:
station_id = df_station.id.tolist()
type(station_id)

list

In [10]:
stations = station_id
measurements=['PM10', 'O3', 'CO', 'NO2', 'SO2', 'WS', 'WD', 'TEMP', 'RH', 'BP', 'RAIN']
sdate='2023-01-01'
edate='2024-01-02'

In [11]:
URL = 'http://air4thai.pcd.go.th/webV2/history/api/data.php?stationID={station_ids}' +  \
     '&param={measurements}&type=hr&sdate={sdate}&edate={edate}'+ \
     '&stime=00&etime=24'
URL

'http://air4thai.pcd.go.th/webV2/history/api/data.php?stationID={station_ids}&param={measurements}&type=hr&sdate={sdate}&edate={edate}&stime=00&etime=24'

In [12]:
station_groups = [stations[i:i+5] for i in range(0, len(stations), 5)]
station_groups

[['02t', '05t', '10t', '11t', '12t'],
 ['59t', '61t', '03t', '50t', '52t'],
 ['53t', '54t']]

In [13]:
records = []
for group in station_groups:
    # Construct the URL for this group of stations
    url = URL.format(station_ids=','.join(group), measurements=','.join(measurements), sdate=sdate, edate=edate)

    # Fetch data from the API
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read().decode())

    # Process the data and add it to the records
    for s in data['stations']:
        for r in s['data']:
            r['DATETIMEDATA'] = records.append(dict(stationID=s['stationID'], **r))

In [14]:
df_Pm10 = pd.DataFrame(records)
df_Pm10

,stationID,DATETIMEDATA,PM10,O3,CO,NO2,SO2,WS,WD,TEMP,RH,BP,RAIN
0,02t,2023-12-01 00:00:00,NaN,NaN,NaN,NaN,None,0.7,49,30.0,53,754,0.0
1,02t,2023-12-01 01:00:00,NaN,NaN,NaN,NaN,None,0.8,51,29.5,55,753,0.0
2,02t,2023-12-01 02:00:00,NaN,NaN,NaN,NaN,None,0.4,5,29.1,57,753,0.0
3,02t,2023-12-01 03:00:00,NaN,NaN,NaN,NaN,None,0.4,30,28.5,60,753,0.0
4,02t,2023-12-01 04:00:00,NaN,NaN,NaN,NaN,None,0.7,19,28.1,62,753,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9511,54t,2024-01-02 20:00:00,67.0,NaN,2.32,NaN,None,0.2,307,30.3,58,760,0.0
9512,54t,2024-01-02 21:00:00,80.0,NaN,2.04,NaN,None,0.1,333,29.9,59,760,0.0
9513,54t,2024-01-02 22:00:00,91.0,NaN,1.66,NaN,None,0.1,358,29.6,59,760,0.0
9514,54t,2024-01-02 23:00:00,78.0,NaN,1.46,NaN,None,0.0,359,29.2,60,760,0.0


In [15]:
df_Pm10['DATETIMEDATA'] = df_Pm10['DATETIMEDATA'].str.slice(stop=16)
df_Pm10

,stationID,DATETIMEDATA,PM10,O3,CO,NO2,SO2,WS,WD,TEMP,RH,BP,RAIN
0,02t,2023-12-01 00:00,NaN,NaN,NaN,NaN,None,0.7,49,30.0,53,754,0.0
1,02t,2023-12-01 01:00,NaN,NaN,NaN,NaN,None,0.8,51,29.5,55,753,0.0
2,02t,2023-12-01 02:00,NaN,NaN,NaN,NaN,None,0.4,5,29.1,57,753,0.0
3,02t,2023-12-01 03:00,NaN,NaN,NaN,NaN,None,0.4,30,28.5,60,753,0.0
4,02t,2023-12-01 04:00,NaN,NaN,NaN,NaN,None,0.7,19,28.1,62,753,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9511,54t,2024-01-02 20:00,67.0,NaN,2.32,NaN,None,0.2,307,30.3,58,760,0.0
9512,54t,2024-01-02 21:00,80.0,NaN,2.04,NaN,None,0.1,333,29.9,59,760,0.0
9513,54t,2024-01-02 22:00,91.0,NaN,1.66,NaN,None,0.1,358,29.6,59,760,0.0
9514,54t,2024-01-02 23:00,78.0,NaN,1.46,NaN,None,0.0,359,29.2,60,760,0.0


In [17]:
df_traffic = pd.read_csv('traffic2023-2024.csv')
df_traffic

,datetime,index
0,2023-01-01 00:00,2.0
1,2023-01-01 00:05,2.6
2,2023-01-01 00:10,1.9
3,2023-01-01 00:15,2.8
4,2023-01-01 00:20,2.5
...,...,...
113219,2024-01-31 23:35,2.1
113220,2024-01-31 23:40,2.3
113221,2024-01-31 23:45,2.7
113222,2024-01-31 23:50,2.6


In [18]:
df_merge = df_Pm10.merge(df_traffic, left_on='DATETIMEDATA', right_on='datetime', how='left')
df_merge

,stationID,DATETIMEDATA,PM10,O3,CO,NO2,SO2,WS,WD,TEMP,RH,BP,RAIN,datetime,index
0,02t,2023-12-01 00:00,NaN,NaN,NaN,NaN,None,0.7,49,30.0,53,754,0.0,2023-12-01 00:00,2.4
1,02t,2023-12-01 01:00,NaN,NaN,NaN,NaN,None,0.8,51,29.5,55,753,0.0,2023-12-01 01:00,2.1
2,02t,2023-12-01 02:00,NaN,NaN,NaN,NaN,None,0.4,5,29.1,57,753,0.0,2023-12-01 02:00,2.5
3,02t,2023-12-01 03:00,NaN,NaN,NaN,NaN,None,0.4,30,28.5,60,753,0.0,2023-12-01 03:00,2.3
4,02t,2023-12-01 04:00,NaN,NaN,NaN,NaN,None,0.7,19,28.1,62,753,0.0,2023-12-01 04:00,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9511,54t,2024-01-02 20:00,67.0,NaN,2.32,NaN,None,0.2,307,30.3,58,760,0.0,2024-01-02 20:00,2.9
9512,54t,2024-01-02 21:00,80.0,NaN,2.04,NaN,None,0.1,333,29.9,59,760,0.0,2024-01-02 21:00,3.3
9513,54t,2024-01-02 22:00,91.0,NaN,1.66,NaN,None,0.1,358,29.6,59,760,0.0,2024-01-02 22:00,2.5
9514,54t,2024-01-02 23:00,78.0,NaN,1.46,NaN,None,0.0,359,29.2,60,760,0.0,2024-01-02 23:00,1.7


In [19]:
df_merge.head(5)

,stationID,DATETIMEDATA,PM10,O3,CO,NO2,SO2,WS,WD,TEMP,RH,BP,RAIN,datetime,index
0,02t,2023-12-01 00:00,NaN,NaN,NaN,NaN,None,0.7,49,30.0,53,754,0.0,2023-12-01 00:00,2.4
1,02t,2023-12-01 01:00,NaN,NaN,NaN,NaN,None,0.8,51,29.5,55,753,0.0,2023-12-01 01:00,2.1
2,02t,2023-12-01 02:00,NaN,NaN,NaN,NaN,None,0.4,5,29.1,57,753,0.0,2023-12-01 02:00,2.5
3,02t,2023-12-01 03:00,NaN,NaN,NaN,NaN,None,0.4,30,28.5,60,753,0.0,2023-12-01 03:00,2.3
4,02t,2023-12-01 04:00,NaN,NaN,NaN,NaN,None,0.7,19,28.1,62,753,0.0,2023-12-01 04:00,1.8


In [20]:
df_merge = df_merge.drop(columns=['datetime'])
df_merge.columns = ['stationID', 'datetime', 'pm10', 'o3', 'co', 'no2', 'so2', 'ws', 'wd', 'temp', 'rh', 'bp', 'rain', 'Traffic congestion index']
df_merge

,stationID,datetime,pm10,o3,co,no2,so2,ws,wd,temp,rh,bp,rain,Traffic congestion index
0,02t,2023-12-01 00:00,NaN,NaN,NaN,NaN,None,0.7,49,30.0,53,754,0.0,2.4
1,02t,2023-12-01 01:00,NaN,NaN,NaN,NaN,None,0.8,51,29.5,55,753,0.0,2.1
2,02t,2023-12-01 02:00,NaN,NaN,NaN,NaN,None,0.4,5,29.1,57,753,0.0,2.5
3,02t,2023-12-01 03:00,NaN,NaN,NaN,NaN,None,0.4,30,28.5,60,753,0.0,2.3
4,02t,2023-12-01 04:00,NaN,NaN,NaN,NaN,None,0.7,19,28.1,62,753,0.0,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9511,54t,2024-01-02 20:00,67.0,NaN,2.32,NaN,None,0.2,307,30.3,58,760,0.0,2.9
9512,54t,2024-01-02 21:00,80.0,NaN,2.04,NaN,None,0.1,333,29.9,59,760,0.0,3.3
9513,54t,2024-01-02 22:00,91.0,NaN,1.66,NaN,None,0.1,358,29.6,59,760,0.0,2.5
9514,54t,2024-01-02 23:00,78.0,NaN,1.46,NaN,None,0.0,359,29.2,60,760,0.0,1.7


In [ ]:
df_merge.to_csv('Merged_PM10_data.csv', index=False)